#### Rough Implementation of  ID3 Algoritm 

In [1]:
import pandas as pd
import numpy as np

#create data for testing out the code

data = [('sunny',    'hot',  'high',   'weak',   'no'),
        ('sunny',    'hot',  'high',   'strong', 'no'),
        ('overcast', 'hot',  'high',   'weak',   'yes'),
        ('rain',     'mild', 'high',   'weak',   'yes'),
        ('rain',     'cool', 'normal', 'weak',   'yes'),
        ('rain',     'cool', 'normal', 'strong', 'no'),
        ('overcast', 'cool', 'normal', 'strong', 'yes'),
        ('sunny',    'mild', 'high',   'weak',   'no'),
        ('sunny',    'cool', 'normal', 'weak',   'yes'),
        ('rain',     'mild', 'normal', 'weak',   'yes'),
        ('sunny',    'mild', 'normal', 'strong', 'yes'),
        ('overcast', 'mild', 'high',   'strong', 'yes'),
        ('overcast', 'hot',  'normal', 'weak',   'yes'),
        ('rain',     'mild', 'high',   'strong', 'no')
       ]

test_df = pd.DataFrame(data, columns = ['outlook', 'temperature', 'humidity', 'wind', 'play tenis'])
test_df.head(3)

,outlook,temperature,humidity,wind,play tenis
0,sunny,hot,high,weak,no
1,sunny,hot,high,strong,no
2,overcast,hot,high,weak,yes


In [2]:
#Get the features and distinguish them examples values to count the classes' targets

feature = test_df.iloc[:,0]
target = test_df.iloc[:,len(test_df.columns)-1]

In [3]:
def find_branches(node):  #return the features /// np.unique would do the trick
    branches = list()
    for i in node:
        if i  not in  branches:
            branches.append(i)
    return branches



In [4]:
def find_root(potential_root,leaves): #finds the root of the tree
    
    f_i = find_branches(potential_root)
    t_i = find_branches(leaves)
    counts = list()
    aux = 0
    
    for i in f_i:
        for j in t_i:
            for k in range(len(potential_root)):
                if(potential_root[k]== i  and leaves[k]== j):
                    aux = aux + 1
            
            counts.append(aux)
            aux = 0  
                
    return counts




### Entropy 

The entropy of a random variable **V** with n values ${v_i}$  , which one with a probality **P($v_i$)** is given by:

$$ H(V) = - \sum_{i = 1}^{n}p(v_i)log_2p(v_i)$$

For a boolean variable, which assumes 2 values, true with probality **q**:

$$B(q) = -(qlog_2q + (1-q)log_2(1-q))$$

After some definitions and manipulations, the information Gain of a goal-atrribute can be defined as:

$$InfoGain(A) = B(\frac{p}{p+n}) - \sum_{i = 1}^{d}\frac{p_i+n_i}{p+n}B(\frac{p_i}{p_i+n_i})$$



In [5]:
def booleantropy(prob):
    
    if(prob == 0 or prob ==1):
        
        return 0 
    else:
        
        return -(((prob*np.log(prob) + (1-prob)*np.log(1-prob))/np.log(2))) 




In [25]:
def infoGain(counts): #sun = 0.970
    pi = []
    ni = []
    aux = 0

    for i in range(len(counts)):
        if(i % 2 == 0 ):
            ni.append(counts[i])
        else:
            pi.append(counts[i])

    n = sum(ni)
    p = sum(pi)

    
    for t in zip(ni,pi):

        aux = aux +  ((t[1] + t[0])/(p+n))*booleantropy(t[1]/(t[1]+t[0])) 
            
    
    return (booleantropy(p/(p+n)) - aux) 

In [26]:
def find_next_node(currently_node,potential_node = False ,leaves = False):
    branches = find_branches(currently_node)
    
    aux_0 = []
    aux_1 = []
    
    for branch in branches:  #get the indexes (sunny, rain and forecast)
        aux_0 = [i for i in range(len(currently_node)) if currently_node[i] == branch] #elegant as fuck dude
        aux_1.append(aux_0)
        aux_0 = [] 

    for element in aux_1:                               # counts the score for each different attibute
        sub_feature = [potential_node[i] for i in element]
        sub_leaves = [leaves[i] for i in element]
        sub_counts  = find_root(sub_feature,sub_leaves)  
        sub_gain = infoGain(sub_counts)
     
    
    

In [8]:
feature = test_df.iloc[:,0]

examples = find_branches(feature)

counts = find_root(feature,target)

print(counts)

infoGain(counts)


[3, 2, 0, 4, 2, 3]


0.2467498197744391

In [29]:
def tree_building(dataset): # dataset -> pandas dataframe. 
    tree = []
    features_idx = list(dataset.columns[0:len(dataset.columns)-1]) # get only the feature labels.
    leaves = dataset.iloc[:,len(test_df.columns)-1]
    
    counts = [find_root(dataset[i],leaves) for i in features_idx]
    gains  = [infoGain(i) for i in counts]    
    tree.append(features_idx[np.argmax(gains)])
    features_idx.remove(tree[0]) #remove the root from the features to be chosen as nodes 
  

    next_el = [find_next_node(dataset[tree[0]], dataset[i], leaves) for i in features_idx]

    
    

In [30]:
tree_building(test_df)


['temperature', 'humidity', 'wind']
